In [2]:
import pandas as pd
import pickle
import tensorflow as tf

from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.callbacks import EarlyStopping

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Input, Embedding, LSTM, Dense, Activation, Conv1D, Flatten, Dropout, MaxPooling1D

import sys
sys.path.append('../../..')
from smartFAQ.src.tokenPad import tokenization_padding

import os

print('Setup is ready')

Setup is ready


# Load Data
---
## Fast Text model

In [3]:
#with open('../../data/fast_text_model_pretrained_weights.pickle', 'rb') as handle:
#    fast_text_model = pickle.load(handle)
with open('../../data/x_train.pickle', 'rb') as handle:
    X_train = pickle.load(handle)
with open('../../data/y_train.pickle', 'rb') as handle:
    y_train = pickle.load(handle)
with open('../../data/x_val.pickle', 'rb') as handle:
    X_val = pickle.load(handle)
with open('../../data/y_val.pickle', 'rb') as handle:
    y_val = pickle.load(handle)

In [4]:
#pretrained_weights = fast_text_model.wv.syn0
#vocab_size, embedding_size = pretrained_weights.shape

## Tokenization & Padding

In [6]:
MAX_NUM_WORDS = 20000


In [7]:
input_tokenizer = Tokenizer(num_words=MAX_NUM_WORDS)
input_tokenizer.fit_on_texts(X_train['answer'])
input_integer_seq = input_tokenizer.texts_to_sequences(X_train['answer'])

fast_text_idx_inputs = input_tokenizer.word_index
print('Total unique words in the input: %s' % len(fast_text_idx_inputs))

max_input_len = max(len(sen) for sen in input_integer_seq)
print("Length of longest sentence in input: %g" % max_input_len)

Total unique words in the input: 30369
Length of longest sentence in input: 1688


In [8]:
x_train_pad = pad_sequences(input_integer_seq, maxlen=max_input_len)

print("encoder_input_sequences.shape:", x_train_pad.shape)
print("encoder_input_sequences[172]:", x_train_pad[72])

encoder_input_sequences.shape: (10000, 1688)
encoder_input_sequences[172]: [  0   0   0 ... 135 183  86]


In [9]:
input_tokenizer = Tokenizer(num_words=MAX_NUM_WORDS)
input_tokenizer.fit_on_texts(X_val['answer'])
input_integer_seq = input_tokenizer.texts_to_sequences(X_val['answer'])
x_val_pad = pad_sequences(input_integer_seq, maxlen=max_input_len)
print("encoder_input_sequences.shape:", x_val_pad.shape)
print("encoder_input_sequences[172]:", x_val_pad[72])

encoder_input_sequences.shape: (10000, 1688)
encoder_input_sequences[172]: [   0    0    0 ... 5837 5419 5838]


## Keras Model

In [11]:
import gensim
from gensim.models import FastText
from gensim.models.phrases import Phrases, Phraser

In [12]:
# sent = []
# for col in ['question', 'answer']:
#     words = [row.split() for row in X_train[col]]
#     sent = sent + words
# sentences = [[word for word in document.lower().split()] for document in X_train['answer']]

In [13]:
#sentences = [[word for word in document.lower().split()] for document in X_train['answer']]

In [14]:
#phrases = Phrases(sentences)
#bigram = Phraser(phrases)
#sentences = bigram[sentences]

In [15]:
# fast_text_model = FastText(min_count=50, window=10, size=300, sample=6e-5, alpha=0.03, min_alpha=0.0007, negative=50, workers=cores-1)

In [16]:
# sentences

In [17]:
with open('../../data/fast_text_model_pretrained_weights.pickle', 'rb') as handle:
    fast_text_model = pickle.load(handle)

In [18]:
pretrained_weights = fast_text_model.wv.syn0
vocab_size, embedding_size = pretrained_weights.shape

C:\Users\achenu\Anaconda3\envs\FAQ\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `syn0` (Attribute will be removed in 4.0.0, use self.vectors instead).
  """Entry point for launching an IPython kernel.


In [19]:
fast_text_model.most_similar(['import'])

C:\Users\achenu\Anaconda3\envs\FAQ\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('import_sys', 0.7419381141662598),
 ('import_os', 0.7078060507774353),
 ('Import', 0.7003344893455505),
 ('import_glob', 0.6803531646728516),
 ('imports', 0.6776682138442993),
 ('imported', 0.6736365556716919),
 ('bs_import', 0.6357274055480957),
 ('flask_import', 0.6279746294021606),
 ('import_threading', 0.6251416206359863),
 ('future_import', 0.6239302754402161)]

In [20]:
def build_model(vocab_size,embedding_size,pretrained_weights):
    
           
    model = tf.keras.Sequential()
    
    model.add(Embedding(input_dim=vocab_size, 
                        output_dim=embedding_size, 
                        weights=[pretrained_weights],
                        input_length=max_input_len
                       ))

    model.add(Conv1D(128,5, activation='relu'))
    model.add(Conv1D(128,5, activation='relu'))
    
    model.add(Flatten())
    model.add(Dropout(.2, input_shape=(2,)))
    
    model.add(Dense(10, activation='relu'))
    model.add(Activation('relu'))
    model.add(Dense(1, activation='sigmoid'))

    return model

In [21]:
# MAE : the average absolute distance between the predicted and target values
def compile_model(model):
    mae = tf.keras.losses.MeanAbsolutePercentageError()
    model.compile(loss=mae,
                  optimizer='adam',
                  metrics=['mean_squared_error', 'mean_absolute_error']
                  )
    return  model

In [22]:
def fit_model( x_train, y_train, x_val, y_val, model, batch_size,  epochs = 5):
    
    print('Train...')
    os.makedirs("./logs/CNN_logs",exist_ok=True)
    tensorboard = TensorBoard(log_dir=os.path.join('./logs/CNN_logs'), histogram_freq=0,
                                  write_graph=True, write_images=False,profile_batch = 100000000)

    # This callback will stop the training when there is no improvement in
    # the validation loss for 2 consecutive epochs.
    Es= EarlyStopping(monitor='loss', patience=2)
    
    callbacks = [Es, tensorboard]

    model.fit(x_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              validation_data=(x_val, y_val),
              callbacks= callbacks)
    return model

In [23]:
with tf.device("/CPU:0"):
    model = build_model(vocab_size, embedding_size, pretrained_weights)
    model.summary()
    model = compile_model(model)
    model = fit_model(x_train_pad, y_train, x_val_pad, y_val, model, batch_size=200, epochs=3)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 1688, 300)         1131600   
_________________________________________________________________
conv1d (Conv1D)              (None, 1684, 128)         192128    
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 1680, 128)         82048     
_________________________________________________________________
flatten (Flatten)            (None, 215040)            0         
_________________________________________________________________
dropout (Dropout)            (None, 215040)            0         
_________________________________________________________________
dense (Dense)                (None, 10)                2150410   
_________________________________________________________________
activation (Activation)      (None, 10)                0

InvalidArgumentError:  indices[6,1674] = 11760 is not in [0, 3772)
	 [[node sequential/embedding/embedding_lookup (defined at <ipython-input-22-1081c3049bf4>:18) ]] [Op:__inference_train_function_1103]

Errors may have originated from an input operation.
Input Source operations connected to node sequential/embedding/embedding_lookup:
 sequential/embedding/embedding_lookup/784 (defined at C:\Users\achenu\Anaconda3\envs\FAQ\lib\contextlib.py:112)

Function call stack:
train_function


## Save model 

In [ ]:
# Save the entire model as a SavedModel.
!mkdir -p saved_model
model.save('./saved_model/CNN_model_FT')

In [ ]:
loss, mean_squared_error, mean_ab = model.evaluate(x_val_pad, y_val)

In [ ]:
# © Laëtitia CONSTANTIN & Axel CHENU 2021